In [63]:
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 데이터 로드
data = pd.read_csv('20171104_test.csv', index_col=0)

# 형태소 분석기 초기화
okt = Okt()

# 기사의 내용을 형태소로 분석
data['tokenized_content'] = data['content'].apply(lambda x: ' '.join(okt.morphs(x)))

# TF-IDF 변환기를 초기화
tfidf_vectorizer = TfidfVectorizer()

# 형태소 분석된 내용을 벡터로 변환
tfidf_matrix = tfidf_vectorizer.fit_transform(data['tokenized_content'])

# 코사인 유사도 행렬 계산
cosine_sim = cosine_similarity(tfidf_matrix)

# 행렬의 통계 값 계산
# Compute statistics for the matrix
min_value = round(np.min(cosine_sim), 4)
max_value = round(np.max(cosine_sim), 4)
mean_value = round(np.mean(cosine_sim), 4)
median_value = round(np.median(cosine_sim), 4)
std_value = round(np.std(cosine_sim), 4)

min_value, max_value, mean_value, median_value, std_value

(0.0127, 1.0, 0.2894, 0.1647, 0.3401)

In [64]:
len(data)

10

In [ ]:
# 군집화
clusters = []
visited = np.zeros(cosine_sim.shape[0], dtype=bool)
for i in range(cosine_sim.shape[0]):
    if not visited[i]:
        similar_articles = np.where(cosine_sim[i] > 0.6)[0]
        clusters.append(similar_articles)
        visited[similar_articles] = True

# 각 군집에서 대표 기사만 남기기
representative_indices = [cluster[0] for cluster in clusters if len(cluster) > 0]

# 대표 기사만 포함된 데이터프레임 생성
filtered_data = data.iloc[representative_indices].reset_index(drop=True)
print(len(filtered_data))
filtered_data

6


,time,title,content,press,link,UTC-time,UTC-date,tokenized_content
0,2017-11-04 09:56:45,잠잠한 북한…트럼프 순방기간 도발 가능성은,\r\n\t\t\t잠잠한 북한…트럼프 순방기간 도발 가능성은[앵커] 북한은 최근 5...,연합뉴스TV언론사 선정,https://n.news.naver.com/mnews/article/422/000...,2017-11-04 00:56:45,2017-11-04,\r\n\t\t\t 잠잠한 북한 … 트럼프 순방 기간 도발 가능성 은 [ 앵커 ] ...
1,2017-11-04 09:01:08,CME 비트코인 거래 개시 소식에 계좌 개설 열풍,\n\n\n\n\n[아시아경제 황준호 기자] 세계 최대 파생상품 거래소인 미국 시...,아시아경제,https://n.news.naver.com/mnews/article/277/000...,2017-11-04 00:01:08,2017-11-04,\n\n\n\n\n [ 아시아 경제 황준호 기자 ] 세계 최대 파생상품 거래소 인 ...
2,2017-11-04 15:37:27,"스파크랩, 1천억 규모 블록체인 투자 펀드 출범",\n\t\t\t 스파크랩 그룹은 블록체인과 암호화 화폐 기업 투자 펀드인 ‘스파크체...,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,2017-11-04 06:37:27,2017-11-04,\n\t\t\t 스파크 랩 그룹 은 블록 체인 과 암호 화 화폐 기업 투자 펀드 ...
3,2017-11-04 19:55:00,"""현직 경찰관에 배우 동원까지""…가상화폐 범죄 기승",\r\n\t\t\t【 앵커멘트 】 하루 거래액이 2조 원을 넘어서면서 비트코인 등...,MBN,https://n.news.naver.com/mnews/article/057/000...,2017-11-04 10:55:00,2017-11-04,\r\n\t\t\t 【 앵커 멘트 】 하루 거래 액 이 2조 원 을 넘어서면서 비트...
4,2017-11-05 01:34:20,"가상화폐, 금융시장의 새로운 쓰나미",\r\n비트코인·이더리움 등 1000종류상위 10개 시가총액만 150조원빠르고 거래...,T19면 TOP,https://n.news.naver.com/mnews/article/353/000...,2017-11-04 16:34:20,2017-11-04,\r\n 비트코인 · 이 더 리움 등 1000 종류 상위 10 개 시가총액 만 15...
5,2017-11-05 06:00:00,[박수찬의 軍]대통령 공약과 따로 노는 국방부의 北 미사일 대책,\r\n\r\n\r\n\r\n\r\n천궁 지대공미사일이 2일 충남 보령 대천사격장에...,세계일보언론사 선정,https://n.news.naver.com/mnews/article/022/000...,2017-11-04 21:00:00,2017-11-04,\r\n\r\n\r\n\r\n\r\n 천궁 지 대 공 미사일 이 2일 충남 보령 대...


In [ ]:
filtered_data.to_csv('filtered_20171104_01.csv', encoding='utf-8-sig', index=False)

In [9]:
import pandas as pd
import torch
from sentence_transformers import util
from transformers import AutoTokenizer, AutoModel
import numpy as np

# 데이터 로드
data = pd.read_csv('20230524_test.csv')

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sbert-multitask')
model = AutoModel.from_pretrained('jhgan/ko-sbert-multitask')

# Mean Pooling 함수 정의
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] 
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# 문장 토크나이즈 및 임베딩 생성
encoded_input = tokenizer(data['content'].tolist(), padding=True, truncation=True, return_tensors='pt')
with torch.no_grad():
    model_output = model(**encoded_input)
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])


# 코사인 유사도 계산 함수
def cosine_similarity(embedding1, embedding2):
    return util.pytorch_cos_sim(embedding1, embedding2)

# 유사도 행렬 초기화 및 계산
similarity_matrix = []
for i in range(len(sentence_embeddings)):
    row = []
    for j in range(len(sentence_embeddings)):
        similarity = cosine_similarity(sentence_embeddings[i], sentence_embeddings[j]).item()
        row.append(similarity)
    similarity_matrix.append(row)

# 행렬의 통계 값 계산
min_value = round(np.min(similarity_matrix), 4)
max_value = round(np.max(similarity_matrix), 4)
mean_value = round(np.mean(similarity_matrix), 4)
median_value = round(np.median(similarity_matrix), 4)
std_value = round(np.std(similarity_matrix), 4)

min_value, max_value, mean_value, median_value, std_value

(0.037, 1.0, 0.4581, 0.4305, 0.1656)

In [10]:
# 군집화
clusters = []
visited = np.zeros(len(data), dtype=bool)
for i in range(len(data)):
    if not visited[i]:
        similar_articles = np.where(np.array(similarity_matrix[i]) > 0.6)[0]
        clusters.append(similar_articles)
        visited[similar_articles] = True

# 각 군집에서 대표 기사만 남기기
representative_indices = [cluster[0] for cluster in clusters if len(cluster) > 0]

# 대표 기사만 포함된 데이터프레임 생성
filtered_data = data.iloc[representative_indices].reset_index(drop=True)
print(len(filtered_data))
filtered_data

48


,date,time,title,content,press,link,UTC-time,UTC-date
0,2023-05-23,2023-05-23 09:50:01,인기 국악인 NFT '혼(魂)' 프로젝트…500개 예약 판매,\n\n\n\n\n국악방송 제공한국의 소리를 NFT(Non-fungible toke...,노컷뉴스,https://n.news.naver.com/mnews/article/079/000...,2023-05-23 00:50:01,2023-05-23
1,2023-05-23,2023-05-23 09:50:02,누리호 3차 발사 성공하니?…'바드'와 '빙'에 물어봤더니,"\n입 모아 ""성공 가능성 커""…구글·MS가 만든 AI 챗봇 비교 체험기'비서 느낌...",연합뉴스,https://n.news.naver.com/mnews/article/001/001...,2023-05-23 00:50:02,2023-05-23
2,2023-05-23,2023-05-23 09:50:08,"윤재옥 ""민주당 강성 지지층 휘둘려 남국 수호 못 벗어나""","\n""김남국 잠적..증거인멸 의심""""즉각 돌아와 檢 수사 협조해야"" \n\n\n\n...",파이낸셜뉴스언론사 선정,https://n.news.naver.com/mnews/article/014/000...,2023-05-23 00:50:08,2023-05-23
3,2023-05-23,2023-05-23 11:19:01,"신한투자증권, MTS ‘알파 3.0’ 출시… “투자 종목, 다른 사람 매수단가도 ...",\n“하반기 알파 4.0 출시 계획도...챗GPT 활용 투자서비스 출격 예정” ...,조선비즈,https://n.news.naver.com/mnews/article/366/000...,2023-05-23 02:19:01,2023-05-23
4,2023-05-23,2023-05-23 11:45:58,[무등의 아침] “노 전 대통령이 여야 간 극한 대립 봤다면…밤샘 토론 제안했...,"\n""노무현 전 대통령이 검찰 정권 탄생 지켜봤다면...'에이, 바보들' 했을 것 ...",KBS,https://n.news.naver.com/mnews/article/056/001...,2023-05-23 02:45:58,2023-05-23
5,2023-05-23,2023-05-23 11:20:13,"이복현 금감원장 ""불공정거래 세력과 전쟁…거취 건다""","\n""선제적 적발 못한 것 사과…신뢰 회복 위해 엄정 대응""(서울=연합뉴스) 오지은...",연합뉴스,https://n.news.naver.com/mnews/article/001/001...,2023-05-23 02:20:13,2023-05-23
6,2023-05-23,2023-05-23 11:15:08,"가상자산거래소 협의체 닥사, 공식 홈페이지 오픈","\n\n\n\n\nDAXA 제공. 국내 5대 가상자산거래소 고팍스, 빗썸, 업...",디지털타임스,https://n.news.naver.com/mnews/article/029/000...,2023-05-23 02:15:08,2023-05-23
7,2023-05-23,2023-05-23 11:18:07,[人사이더] 결백하다더니 잠적한 김남국… ‘수사회피·증거인멸’ 의심한 윤...,\n\n\n\n\n윤재옥 국민의힘 원내대표가 23일 국회에서 열린 원내대책회의에서 ...,디지털타임스,https://n.news.naver.com/mnews/article/029/000...,2023-05-23 02:18:07,2023-05-23
8,2023-05-23,2023-05-23 11:42:04,윤재옥 “대선 직전 돈세탁 의심…오는 25일 ‘김남국 방지법’ 통과”,"\n“김남국, 대선 직전 자금 세탁하고 현금 대량 인출해…합리적 의심” ...",조선비즈,https://n.news.naver.com/mnews/article/366/000...,2023-05-23 02:42:04,2023-05-23
9,2023-05-23,2023-05-23 12:50:03,"與 코인게이트 조사단 ""마브렉스, 김남국 사전정보 취득 가능성 공감""","\n""코인 중독자 김남국, 정치적 심판서 법적 의혹으로 확대""""마브렉스, 자체 조사...",뉴스1,https://n.news.naver.com/mnews/article/421/000...,2023-05-23 03:50:03,2023-05-23


In [6]:
filtered_data['content'][25]

'\n\'가상자산 4대 공약\' 발표…시장 법제화·투자자 보호 강조\n\n\n\n이재명 더불어민주당 대선 후보가 \'가상자산 법제화\'를 추진해 입법 공백을 메우겠다며 이를 포함한 \'가상자산 4대 공약\'을 발표했다. 지난해 11월 11일 국회 의원회관에서 열린 가상자산 간담회 모습. /남윤호 기자이재명 더불어민주당 대선 후보가 가상자산시장을 규제해왔던 민주당 기조에 대해 사과하면서 가상자산 법제화 공약을 발표했다. 그는 현재 가상자산 과세 기준을 현행 250만 원에서 상향해야 한다고 했다.이 후보는 19일 오전 서울 강남구에서 가상자산 4대 거래소 대표 및 전문가들과 가진 간담회에서 "가상자산시장에 적극적으로 접근할 필요가 있다"고 말했다.그는 "가상자산시장이라고 하는 건 우리가 부정해서 없어지는 거라면 부정할 수도 있고 금지할 수도 있겠지만 이미 전 세계에서 실제로 존재하는 시장인데 이 점을 우리가 외면한다고 해서 없어지지도 않고 오히려 기회만 잃게 된다는 생각을 하게 된다"며 "관련 산업도 계속 확대되면 좋겠고, 거래에 참여하는 국민도 보호하는 방향으로 가면 좋겠다. 제도화하고 제도 안에서 부작용을 최소화하고 거기에서 가능성을 찾아내는 게 우리가 갈 길"이라고 했다.이 후보는 특히 \'부동산 기반 가상자산\'에 대한 구상도 거듭 밝혔다. 이 후보는 그동안 부동산 개발 불로소득 국가 환수를 위해 블록체인에 기반을 둔 전 국민 개발이익 공유시스템을 검토하고 있다고 언급해온 바 있다.그는 이날도 "저는 실제로 해보고 싶은데 반론이 엄청 많다"면서 "우리가 꿈꾸는 불로소득 전 국민 환수, 전국민 공유를 하려면 대규모 개발에 참여할 기회를 전 국민에 주고 이를 가상자산화하자는 것"이라고 설명했다. 예를 들어 특정 지역의 개발사업에 50조 원의 투자금이 필요한 경우 가상자산을 통해 전 국민에 100만 원씩 투자할 기회를 준다는 구상이다. 이 후보는 "국민주와 마찬가지로 부동산 대규모 개발에 참여할 수 있는 권리를 주고, 사업의 일정 규모 이상은 자금 조달을 그런 

In [11]:
filtered_data.to_csv('ko_sbert_20230524.csv', encoding='utf-8-sig', index=False)